## Εργασία 3 ##

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [ ]:
# Import from sklearn dataset for breast cancer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

random_state = 0
# Split the data into training and test sets with 70% training and 30% test data
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)


In [ ]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):

        #Initializing the class parameters
        self.estimator = estimator
        self.models = []
        self.used_features = []

    def fit(self, X_train, y_train):

        n_features = X_train.shape[1]

        for i in range(n_features):
            #Creating a new dataset deleting the i-th element
            X_new = np.delete(X_train, i, axis=1)

            #Cloning the initial estimator
            model = clone(self.estimator)

            #Training the model in the new training set
            model.fit(X_new, y_train)

            #Storing the new model and features
            self.models.append(model)
            self.used_features.append(np.delete(np.arange(n_features), i))

    def predict(self, X):

        predictions = np.zeros((X.shape[0], len(self.models)))

        for i, model in enumerate(self.models):
            #Creating a new set deleting the i-th element
            X_new = np.delete(X, i, axis=1)

            #Making the predictions
            predictions[:, i] = model.predict(X_new)

        #Getting the final prediction
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

        return final_predictions

In [ ]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.metrics import accuracy_score

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators
        self.estimator = estimator

        self.estimators = None
        self.estimator_weights = None

        self.total_errors = None

    def fit(self, X_train, y_train):

        self.estimators = []
        self.estimator_weights = []
        self.total_errors = []

        weights = np.full(len(X_train), 1/len(X_train))

        for est_i in range(self.n_estimators):
            #Cloning the basic estimator and training it using the current train set
            estimator = clone(self.estimator)
            estimator.fit(X_train, y_train, weights)

            #Making predictions and calculating the error
            prediction = estimator.predict(X_train)
            error = np.where(prediction != y_train, weights, 0).sum()

            #Calculating the alpha using the error
            alpha = 0.5 *np.log((1 - error)/error)

            #Saving the estimator and the alpha
            self.estimators.append(estimator)
            self.estimator_weights.append(alpha)

            #Updating the weights
            weights = np.where(prediction != y_train, weights * np.exp(alpha), weights * np.exp(-1 * alpha))

            #Normalizing the weights so that they sum up to 1
            weights = weights / weights.sum()

            #Saving the error
            self.total_errors.append(error)

    def predict(self, X):
        predictions = np.stack([estimator.predict(X) for estimator in self.estimators], axis=1)
        weighted_majority_vote = lambda x: np.unique(x)[np.argmax([np.where(x==categ, self.estimator_weights, 0).sum() for categ in np.unique(x)])]
        return np.apply_along_axis(weighted_majority_vote, axis=1, arr=predictions)

In [ ]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [ ]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591